In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from utils import *

In [3]:
root_img = '/media/scratch/astamoulakatos/NMIS/images/'
root_annotation = '/media/scratch/astamoulakatos/NMIS/exported-labels/img-all/Annotations/'
df = generate_df(root_annotation, root_img)

In [4]:
df.head()

,filename,width,height,class,bboxes
0,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[1494, 434, 1694, 633], [396, 1137, 591, 1332..."
1,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[406, 374, 611, 568], [1499, 384, 1694, 578]]"
2,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[726, 859, 925, 1054], [1699, 2301, 1893, 249..."
3,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[217, 2465, 416, 2665], [227, 265, 431, 459],..."
4,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[1414, 429, 1614, 623]]"


In [5]:
df = df.dropna()

In [6]:
df = df.reset_index(drop=True)

In [7]:
len(df)

184

In [8]:
def bolts_count(x):
    return int(str(x).split('_')[1])

In [9]:
df['total_bolts'] = df.filename.apply(bolts_count)

In [10]:
df.head()

,filename,width,height,class,bboxes,total_bolts
0,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[1494, 434, 1694, 633], [396, 1137, 591, 1332...",5
1,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[406, 374, 611, 568], [1499, 384, 1694, 578]]",2
2,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[726, 859, 925, 1054], [1699, 2301, 1893, 249...",7
3,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[217, 2465, 416, 2665], [227, 265, 431, 459],...",5
4,/media/scratch/astamoulakatos/NMIS/images/2021...,4032,3024,Bolt,"[[1414, 429, 1614, 623]]",1


In [11]:
df.to_csv('dataset.csv', index=False)